In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, minimize
import os

# Configure LaTex in Matplotlib
plt.rcParams.update({
    'text.usetex': True,  # Use LaTex for text rendering
    'font.family': 'serif',  # Use a serif font for a more classic LaTex look
    'font.serif': ['Computer Modern Roman'],  # Default LaTex font
    'font.size': 15,  # Base font size for all text elements
    'axes.titlesize': 16,  # Font size for axis titles
    'axes.labelsize': 15,  # Font size for axis labels
    'xtick.labelsize': 15,  # Font size for x-axis tick labels
    'ytick.labelsize': 15,  # Font size for y-axis tick labels
    'legend.fontsize': 15  # Font size for legend text
    })

In [95]:
file_path = r"data\values obtained by Nova2.1 integrited fitting.xlsx"
res = pd.read_excel(file_path).iloc[:, [0, 1]]  

for i in range(1, 31):
    file_name = f"sample {i}.xlsx"
    file_path = os.path.join('data', file_name)

    df = pd.read_excel(file_path).iloc[:, [2, 3]]    
    df_name = f"df_{i}"

    globals()[df_name] = pd.DataFrame({'x': df.iloc[:, 0],
                                       'y': df.iloc[:, 1],})


In [93]:
def zero_grad(df, tol=1e-5, index=False, shift=8):
    # Compute gradient
    dx = df['x'].iloc[1] - df['x'].iloc[0]
    dy = np.gradient(df['y'], dx)

    # Find where first derivative is approximately zero (remove any from start of dataset)
    zero_indices = np.where(np.abs(dy) < tol)[0]
    zero_indices = [idx for idx in zero_indices if idx > 25]

    # Find instances of zero gradient (x,y)
    candidates = [(df['x'].iloc[idx], df['y'].iloc[idx]) for idx in zero_indices]

    # Recursively increase tolerance if no candidates found
    if not candidates and tol < 10:
        return zero_grad(df, tol * 1.2, index)
    
    # Once point found, either return index or value
    else:
        if index:
            return zero_indices[0] - shift
        else:
            return candidates[0] if candidates else None


# Define the equation of a circle for fitting
def circle_equation(params, x, y):
    xc, yc, R = params
    return (x - xc)**2 + (y - yc)**2 - R**2


# Define the objective function for the optimization
def objective_function(params, x, y):
    return np.sum(circle_equation(params, x, y)**2)


def get_result_graph(result, x_data, y_data):
    # Extract optimized parameters
    xc_opt, yc_opt, R_opt = result.x

    # Create a figure and an axis
    fig, ax = plt.subplots(figsize=(8, 4))  # Create a new figure with given size

    # Scatter plot for data points
    ax.scatter(x_data, y_data, color='navy', s=40, alpha=0.7, label='Experimental Data')  # Slight opacity

    # Fitted circle
    circle = plt.Circle((xc_opt, yc_opt), R_opt, color='crimson', fill=False, linestyle='--', linewidth=2, label='Fitted Circle')
    ax.add_patch(circle)  # Use add_patch to add custom shapes like Circle

    ax.set_xlabel(r"$x$")  # LaTex-style axis label
    ax.set_ylabel(r"$y$")

    # Legend
    ax.legend(loc='upper center', bbox_to_anchor=(0.45, 1.15), ncol=5, frameon=False)  # No box around the legend

    # Set x and y axis limits
    ax.set_xlim([0, max(x_data)*1.1])
    ax.set_ylim([0, max(y_data)*1.1])

    # Set ticks and grid
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left') 
    ax.grid(True, linestyle='--', alpha=0.5)  
    ax.set_aspect('equal', 'box') 
    fig.tight_layout()
    
    # Save as high-quality PNG with high DPI
    output_filename = "res_plot.png"
    fig.savefig(output_filename, dpi=300, bbox_inches='tight') 
    plt.show()
    
def get_impedance_eq(df, default_z_cross=True, graph=False):
    # Use zero_grad() to split dataset to circular region
    if default_z_cross:
        z_cross = zero_grad(df, tol=1e-7, index=True)
    else:
        z_cross = -24
    
    x_data = df['x'].iloc[:z_cross]
    y_data = df['y'].iloc[:z_cross]

    # Initial guess for parameters (center and radius) + minimization
    params_guess = [np.mean(x_data), np.mean(y_data), (max(x_data) - min(x_data)) / 2]
    result = minimize(objective_function, params_guess, args=(x_data, y_data))

    # Extract optimized parameters
    xc_opt, yc_opt, R_opt = result.x
    
    if graph: 
        get_result_graph(result, x_data, y_data)
    #print("Optimized parameters (xc, yc, R):", [xc_opt, yc_opt, R_opt])
    #print(f"Impedance: {R_opt*2:.3g} Ω")
    return(R_opt*2)

def get_final_impedance(df, graph = False):
    res = get_impedance_eq(df, graph=False)
    if res > 1000:
        D_opt = get_impedance_eq(df, default_z_cross=False)
        return(D_opt)
    
    else:
        return(res)

In [94]:
imp_calc = []
for k in range(1,31):
    #print(f"Sample {k}")
    holder = get_final_impedance(globals()[f"df_{k}"])
    imp_calc.append(holder)

In [98]:
# Examining Results
res['Calc Impedance'] = imp_calc
res['Abs Error'] = (res['Impedance(Ohms)'] - res['Calc Impedance'])
res['Percentage Error'] = ((res['Impedance(Ohms)'] - res['Calc Impedance'])/res['Impedance(Ohms)']) *100
MAE = sum(res['Abs Error'])/30
print(f'Mean Absolute Error: {MAE:.2f}')

mape = res['Percentage Error'].abs().mean()
print(f'Mean Absoute % Error: {mape:.2f}')

Mean Absolute Error: 1.60
Mean Absoute % Error: 0.72


In [100]:
res

,Sample No.,Impedance(Ohms),Calc Impedance,Abs Error,Percentage Error
0,1,2030,2023.381388,6.618612,0.326040
1,2,1710,1714.552404,-4.552404,-0.266222
2,3,1400,1411.362926,-11.362926,-0.811638
3,4,2170,2174.373864,-4.373864,-0.201561
4,5,1900,1880.585626,19.414374,1.021809
5,6,1790,1785.228134,4.771866,0.266585
6,7,2240,2248.940183,-8.940183,-0.399115
7,8,1870,1825.201490,44.798510,2.395642
8,9,1930,1919.228059,10.771941,0.558132
9,10,2300,2270.643588,29.356412,1.276366
